# Laboratorio 3


## Integrantes
- Gustavo Cruz 22779
- Mathew Cordero 22982
- Pedro Guzmán 22111

## Repositorio

[Link al Repositorio](https://github.com/donmatthiuz/ModelacionSimu/tree/lab3)




## Ejercicio 1  

Implementar los siguientes métodos de **descenso gradiente** (naïve = tamaño de paso α constante):  

1. Descenso gradiente naïve con dirección de descenso **aleatoria**.  
2. Descenso gradiente **máximo** naïve.  
3. Descenso gradiente de **Newton**, con Hessiano exacto.  
4. Un método de **gradiente conjugado** (Fletcher-Reeves, Hestenes-Stiefel o Polak-Ribiere).  
5. El método **BFGS**.  

### Requerimientos

En cada uno de los métodos, la función debe recibir los siguientes argumentos:

- La función objetivo `f`.  
- El gradiente de la función objetivo `df`.  
- El Hessiano `ddf` (cuando sea necesario).  
- Un punto inicial `x0 ∈ R^n`.  
- El tamaño de paso `α > 0`.  
- El número máximo de iteraciones `maxIter`.  
- La tolerancia `ε`.  
- Un criterio de paro.  

### Resultados esperados

Los algoritmos deben devolver:  

- La mejor solución encontrada `best` (la última de las aproximaciones calculadas).  
- La secuencia de iteraciones `xk`.  
- La secuencia de valores `f(xk)`.  
- La secuencia de errores en cada paso (según el error de su criterio de paro).  

Además, es deseable indicar:  

- El número de iteraciones efectuadas por el algoritmo.  
- Si se obtuvo o no la **convergencia** del método.  


Lo primero que vamos a hacer es definir una clase con funciones que representan cada metodo|

In [6]:

import numpy as np

def numerical_derivative(f, x, h=1e-6):
    return (f(x + h) - f(x)) / h

class DescensoGradiente:
    def __init__(self, F, df):
        self.F = F     
        self.df = df   

    def naive_random(self, x0 = np.zeros(2), alpha = 0.01, maxIter = 10000, tol = 1e-10):
        xk = [x0]             
        fk = [self.F(x0)]         # valores f(xk)
        errors = [np.linalg.norm(self.df(x0))] if self.df else [np.inf]

        x = np.array(x0, dtype=float)
        for k in range(maxIter):
            # Generar dirección aleatoria normalizada
            d = np.random.randn(*x.shape)
            d = d / np.linalg.norm(d)

            # Nuevo candidato
            x_new = x - alpha * d
            f_new = self.F(x_new)

            # Guardar información
            xk.append(x_new)
            fk.append(f_new)
            error = np.abs(fk[-1] - fk[-2])  # criterio basado en mejora
            errors.append(error)

            # Condiciones de paro
            if error < tol:
                return x_new, xk, fk, errors, k+1, True

            # Actualizar
            x = x_new

        return x, xk, fk, errors, maxIter, False
    
    def steepest_descent(self, x0 = np.zeros(2), alpha = 0.01, maxIter = 10000, tol = 1e-10): 
        xk = [x0]             
        fk = [self.F(x0)]     # valores f(xk)
        errors = [np.linalg.norm(self.df(x0))]

        x = np.array(x0, dtype=float)
        for k in range(maxIter):
            grad = self.df(x)

            # Condición de paro: norma del gradiente
            if np.linalg.norm(grad) < tol:
                return x, xk, fk, errors, k, True

            # Actualización este es el corzaon de nuestra funcion xk+1​=xk​−αdk​
            x_new = x - alpha * grad
            f_new = self.F(x_new)

            # Guardar información
            xk.append(x_new)
            fk.append(f_new)
            error = np.abs(fk[-1] - fk[-2]) 
            errors.append(error)

            if error < tol:
                return x_new, xk, fk, errors, k+1, True

            x = x_new

        return x, xk, fk, errors, maxIter, False
    
    def newton_method(self, ddf, x0=np.zeros(2), maxIter=10000, tol=1e-10):
        xk = [x0]
        fk = [self.F(x0)]
        errors = [np.linalg.norm(self.df(x0))]
        x = np.array(x0, dtype=float)

        for iteration in range(maxIter):
            grad = self.df(x)
            H = ddf(x)  # Hessiano en x

          
            if np.linalg.norm(grad) < tol:
                print("Converged (gradiente) after", iteration, "iterations.")
                return x, xk, fk, errors, iteration, True

            # Resolver sistema lineal para delta_x
            try:
                delta_x = np.linalg.solve(H, -grad)
            except np.linalg.LinAlgError:
                print("Hessian is singular!")
                return x, xk, fk, errors, iteration, False

            x_new = x + delta_x
            f_new = self.F(x_new)

            # Guardar info
            xk.append(x_new)
            fk.append(f_new)
            errors.append(np.linalg.norm(delta_x))

            # Condición de paro
            if np.linalg.norm(delta_x) < tol:
                print("Converged (delta_x) after", iteration + 1, "iterations.")
                return x_new, xk, fk, errors, iteration + 1, True

            x = x_new

        return x, xk, fk, errors, maxIter, False

    def conjugate_gradient_flerev(self, x0=np.zeros(2), alpha=0.01, maxIter=1000, tol=1e-10):
        xk = [np.array(x0, dtype=float)]
        fk = [self.F(x0)]
        errors = [np.linalg.norm(self.df(x0))]

        x = np.array(x0, dtype=float)
        grad = self.df(x)
        d = -grad  # primera dirección

        for k in range(maxIter):
            grad_old = grad.copy()

            # Actualizar x
            x_new = x + alpha * d
            grad = self.df(x_new)

            # Guardar info
            xk.append(x_new)
            fk.append(self.F(x_new))
            errors.append(np.linalg.norm(grad))

            # Condición de paro
            if np.linalg.norm(grad) < tol:
                return x_new, xk, fk, errors, k+1, True

            # Calcular beta
            beta = np.dot(grad, grad) / np.dot(grad_old, grad_old)
            

            # Actualizar dirección
            d = -grad + beta * d
            x = x_new

        return x, xk, fk, errors, maxIter, False
    
    def bfgs_method(self, x0=np.zeros(2), tol=1e-6, maxIter=1000):
        x = np.array(x0, dtype=float)
        n = len(x)
        H = np.eye(n)  # Aproximación inicial de Hessiano inverso
        xk = [x.copy()]
        fk = [self.F(x)]
        errors = [np.linalg.norm(self.df(x))]

        for k in range(maxIter):
            grad = self.df(x)
            # Dirección de búsqueda
            d = - H.dot(grad)
            x_new = x + d  # Actualización del punto
            grad_new = self.df(x_new)

            # Guardar información
            xk.append(x_new.copy())
            fk.append(self.F(x_new))
            errors.append(np.linalg.norm(grad_new))

            # Condición de paro
            if np.linalg.norm(x_new - x) < tol:
                return x_new, xk, fk, errors, k+1, True

            # Actualizar Hessiano inverso con la fórmula BFGS
            s = x_new - x
            y = grad_new - grad
            rho = 1.0 / np.dot(y, s)
            Hy = H.dot(y)
            H += np.outer(s, s) * rho - np.outer(Hy, Hy) / np.dot(y, Hy)

            # Preparar siguiente iteración
            x = x_new

        return x, xk, fk, errors, maxIter, False
